In [5]:
# コンテスト中に解いた解き方
# 長さ3の巡回置換をしまくる

import heapq
from collections import defaultdict

def main():
    N = int(input())
    AN = [*map(int, input().split())]
    BN = [*map(int, input().split())]
    B_dict = defaultdict(list)
    if sorted(AN) != sorted(BN):
        print("No")
        return
    if len(set(AN)) != len(AN):
        print("Yes")
        return
    for i, B in enumerate(BN):
        heapq.heappush(B_dict[B], i)
    for i in range(N-2):
        if AN[i] == BN[i]:
            heapq.heappop(B_dict[BN[i]])
            continue
        if B_dict[AN[i]] == list():
            print("No")
            return
        Ai_ind = heapq.heappop(B_dict[AN[i]])
        tmp_ind = N-1
        if Ai_ind == tmp_ind:
            tmp_ind -= 1
        B_dict[BN[i]].remove(i)
        B_dict[BN[tmp_ind]].remove(tmp_ind)
        heapq.heappush(B_dict[BN[i]], tmp_ind)
        heapq.heappush(B_dict[BN[tmp_ind]], Ai_ind)
        BN[i], BN[Ai_ind], BN[tmp_ind] = BN[Ai_ind], BN[tmp_ind], BN[i]
    if AN[-1] == BN[-1] and AN[-2] == BN[-2]:
        print("Yes")
    else:
        print("No")
    
if __name__ == "__main__":
    main()

3
1 2 1
1 1 2
Yes


In [5]:
# 転倒数調べるバージョン

class BIT:
    def __init__(self, n):
        self.n = len(n) if isinstance(n, list) else n
        self.size = 1 << (self.n - 1).bit_length()
        if isinstance(n, list):  # nは1-indexedなリスト
            a = [0]
            for p in n: a.append(p + a[-1])
            a += [a[-1]] * (self.size - self.n)
            self.d = [a[p] - a[p - (p & -p)] for p in range(self.size + 1)]
        else:                    # nは大きさ
            self.d = [0] * (self.size + 1)

    def __repr__(self):
        p = self.size
        res = []
        while p > 0:
            res2 = []
            for r in range(p, self.size + 1, p * 2):
                l = r - (r & -r) + 1
                res2.append(f'[{l}, {r}]:{self.d[r]}')
            res.append(' '.join(res2))
            p >>= 1
        res.append(f'{[self.sum(p + 1) - self.sum(p) for p in range(self.size)]}')
        return '\n'.join(res)

    def add(self, p, x):  # O(log(n)), 点pにxを加算
        assert p > 0
        while p <= self.size:
            self.d[p] += x
            p += p & -p

    def get(self, p, default=None):     # O(log(n))
        assert p > 0
        return self.sum(p) - self.sum(p - 1) if 1 <= p <= self.n or default is None else default

    def sum(self, p):     # O(log(n)), 閉区間[1, p]の累積和
        assert p >= 0
        res = 0
        while p > 0:
            res += self.d[p]
            p -= p & -p
        return res

    def lower_bound(self, x):   # O(log(n)), x <= 閉区間[1, p]の累積和 となる最小のp
        if x <= 0: return 0
        p, r = 0, self.size
        while r > 0:
            if p + r <= self.n and self.d[p + r] < x:
                x -= self.d[p + r]
                p += r
            r >>= 1
        return p + 1

# 転倒数 (BIT必須, O(NlogN)) BITは1-indexedであることに注意する
# V: 転倒数を求めたい1-indexedの数列のリスト 例 [3,2,1]なら転倒数は3
def count_inversions(V):
    bit = BIT(max(V))
    ans = 0
    for i, v in enumerate(V):
        ans += i - bit.sum(V[i])
        bit.add(v, 1)
    return ans

def main():
    N = int(input())
    AN = [*map(int, input().split())]
    BN = [*map(int, input().split())]
    if sorted(AN) != sorted(BN):
        print("No")
        return
    if len(AN) != len(set(AN)):
        print("Yes")
        return
    if count_inversions(AN) % 2 == count_inversions(BN) % 2:
        print("Yes")
    else:
        print("No")
    
if __name__ == "__main__":
    main()

8
1 2 3 4 5 6 7 8
7 8 5 6 4 3 1 2
No
